In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
import random
from scipy import stats
import plotly.express as px
from sklearn.metrics import mean_squared_error, r2_score

import os
from pipeline_config import *


In [3]:
Lr

0.00019837551214953915

In [4]:
#Seeds for reproducability:

torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

In [5]:
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Norway/basic/test_predictions.pickle', 'rb') as f:
    prediction_dic = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/pickle_files/bi_directional_norway_dataset_dict.pickle', 'rb') as f:
    raw_dataset_dict = pickle.load(f)
    
with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

In [6]:
def create_test_dataframe(raw_dataset_dict: dict):
    test_dict = {}
    for key in raw_dataset_dict.keys():
    #for key in ['All items']:
        df = raw_dataset_dict[key][['Category', 'Date', 'Year', 'Indent', 'Inflation t+1']]
        df.dropna(inplace=True)
        df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
        target_df = df[df['Year'] > Year]
        test_dict[key] = target_df
    return test_dict

test_dict = create_test_dataframe(raw_dataset_dict)


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_69432/1267119108.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_69432/1267119108.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_69432/1267119108.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

In [7]:
def get_df_with_predictions(prediction_dic: dict, dict_of_categories_df:dict) -> dict:
    all_data_dict = {}
    for key in list(prediction_dic.keys()):
        predictions = prediction_dic[key]
        prediction_df =  pd.DataFrame(predictions.flatten().detach().numpy())
        prediction_df.rename(columns = {0: 'Prediction'}, inplace=True)
        dict_of_categories_df[key] = dict_of_categories_df[key].reset_index(drop=True)
        all_data_dict[key] = pd.concat([dict_of_categories_df[key], prediction_df], axis=1)
    return all_data_dict

all_data_test_dict=get_df_with_predictions(prediction_dic, test_dict)

In [8]:
# sanity check
all_categories = list(all_data_test_dict.keys())
len(all_categories)

52

Average RMSE:

In [9]:
def avg_rmse(all_data_test_dict):
    mse_lst = []
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)
        mse_lst.append(mse)
    
    rmse_list = list(map(np.sqrt,mse_lst))
    avg_rmse = np.mean(rmse_list)
    rmse_std = np.std(rmse_list)
    
    print(f'RMSE:  {avg_rmse}')
    print(f'MSE std:  {rmse_std}')
    print(f'interval: {[avg_rmse-rmse_std, avg_rmse+rmse_std]}')

    return avg_rmse,rmse_std

avg_rmse(all_data_test_dict)

RMSE:  1.512775462336573
MSE std:  1.3801406156974252
interval: [0.13263484663914782, 2.8929160780339984]


(1.512775462336573, 1.3801406156974252)

Headline RMSE:

In [10]:
df_predictions = all_data_test_dict['All-items']
y_pred = df_predictions['Prediction'].values
y_actual = df_predictions['Actual'].values
mse = mean_squared_error(y_pred, y_actual)
rmse = np.sqrt(mse)


print(f'rmse: {rmse}')

rmse: 0.5794358797024644


In [11]:
print(f'y_pred: {y_pred}')
print(f'y_actual: {y_actual}')

y_pred: [ 1.8658319e-01  2.1595535e-01  2.9726282e-01  1.7738479e-01
  1.7273682e-01  3.0923989e-01 -1.8259369e-02  2.9938352e-01
  2.0793900e-01  2.5581259e-02  4.6642077e-01  3.4230724e-01
  1.6660485e-01  3.6024302e-04  3.3838558e-01  1.6502243e-01
  2.8592765e-01  3.9128724e-01  1.4176836e-02  4.5519832e-01
 -2.3186505e-02  3.6892387e-01  2.7595967e-01 -1.4060789e-01
  7.3407698e-01  1.6105349e-01  4.3915132e-01  1.2859426e-01]
y_actual: [ 0.6986928  -0.26143806  0.34843241 -0.08699435  0.34752424  0.86356322
  0.          1.02652741 -0.25564564  0.76498461  0.67510805 -0.92945325
  1.09752051  0.58602093  1.1618388   0.24721891  0.90127607  1.2966146
 -0.24183809  1.36274654  0.31796529 -0.15885627  0.07945968  0.15873019
  0.39572667  0.78678612  1.09120335  0.46403796]


Total Correlation:

In [12]:
def total_corr(all_data_test_dict):
    corr_dict = {}
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        corr = stats.pearsonr(y_pred,y_actual)[0]
        corr_dict[key] =  corr
    
    total_corr = sum(corr_dict.values())
    
    num_high_corr = 0
    for category in corr_dict:
        if corr_dict[category] >= 0.5:
            num_high_corr +=1
    
    print(f'Number of categories with High Correlation: {num_high_corr}')
    
    return total_corr

total_corr(all_data_test_dict)

Number of categories with High Correlation: 17


18.449395197001493

Average R2:

In [13]:
def avg_r_squared(all_data_test_dict):
    r_squared_lst = []

    for key in all_categories:
        #print(f'category: {key}')
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        r2 = r2_score(y_actual, y_pred)
        r_squared_lst.append(r2)

        if key =='All-items':
            headline_r2 = r2
    
    avg_r2_score = np.mean(r_squared_lst)
    r2_std = np.std(r_squared_lst)
    
    print(f'Average R Squared:  {avg_r2_score}')
    print(f'R Squared std:  {r2_std}')
    print('--------------------------------------------------------')
    print(f'Headline R2: {headline_r2}')
    print(f'R2 list percentiles:\n[10: {np.percentile(r_squared_lst, 10)}, 25: {np.percentile(r_squared_lst, 25)}, 50: {np.percentile(r_squared_lst, 50)}, 75: {np.percentile(r_squared_lst, 75)}, 90: {np.percentile(r_squared_lst, 90)}]')

    return headline_r2, avg_r2_score

headline_r2, avg_r2_score = avg_r_squared(all_data_test_dict)

Average R Squared:  0.09030062144840792
R Squared std:  0.31665832858745546
--------------------------------------------------------
Headline R2: -0.10743444204765806
R2 list percentiles:
[10: -0.235832503153702, 25: -0.06736992531526381, 50: 0.01613788315134157, 75: 0.24605151305417308, 90: 0.5320529309100428]


Plot Results:

In [14]:
def plot_results(all_data_dict, categories):
    category_samples = ['All-items']+random.sample(categories, 10)
    for category in category_samples:
        category_df = all_data_dict[category]
        fig = px.line(category_df, x="Date", y=["Actual", "Prediction"], title=f'{category} - Actual VS Prediction')
        fig.show()

plot_results(all_data_test_dict, all_categories)